```
bryan@ichabod mac_bucket % for file in *_formation.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/formation/
done
```

In [ ]:
df = spark.read.format('json').load("/Volumes/geodata/petra/formation/")

display(df)

In [ ]:
from common.transforms import string_to_iso_date, generate_hash
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("well.uwi").alias("uwi"),
    F.col("well.wsn").alias("wsn"),
    F.col("zdata.z").alias("depth"),
    F.col("zflddef.desc").alias("description"),
    # F.col("zflddef.fid").alias("fid"),
    F.col("zflddef.name").alias("name"),
    F.col("zflddef.source").alias("source"),
    F.col("zflddef.adddate").alias("app_row_created"),
    F.col("zflddef.chgdate").alias("app_row_changed")
)

display(df_flat)

date_columns = [
    "app_row_created",
    "app_row_changed"
]

df_formation = df_flat

# enforce timestamp for dates
for col_name in date_columns:
    df_formation = string_to_iso_date(df_formation, col_name, col_name)

# add id hash
# NOTE: This is just a unique repo+well, not unique formations (arrays)
id_columns = ["repo_id", "uwi"]
df_formation = generate_hash(df_formation, "id", "formation", *id_columns)


display(df_formation)